In [1]:
%matplotlib inline
import numpy as np
from scipy import stats
from blimpy import Waterfall
from blimpy.utils import rebin
from matplotlib import pyplot as plt
from bisect import bisect_left
from tqdm import tqdm
import dask.array as da
import h5py
from time import time

from utils import *

fil_path = "data/filterbanks/"
h5_path = "data/h5/"

test_fil = fil_path + "blc20_guppi_57991_48899_3C161_0007.gpuspec.0000.fil"

fri_obs = h5_path + "GBT_57532_09539_HIP56445_fine.h5"

plt_args = {
            'aspect':'auto',
            'origin':'lower',
            'rasterized':True,
            'interpolation':'nearest',
            'cmap':'viridis'
            }
try:
    client.close()
except NameError:
    pass
from dask.distributed import Client, progress
client = Client(processes=False, threads_per_worker=3,
                n_workers=4, memory_limit='4GB')
client

Client Scheduler: inproc://10.0.0.32/43036/1 Dashboard: http://10.0.0.32/43036/1:8787/status,Cluster Workers: 4 Cores: 12 Memory: 16.00 GB


In [2]:
# Show Info
wf = Waterfall(fri_obs, load_data=False)
wf.info()

blimpy.file_wrapper INFO     Skipping loading data ...

--- File Info ---
b'DIMENSION_LABELS' : [b'frequency' b'feed_id' b'time']
     b'az_start' :                              0.0
    b'data_type' :                                1
         b'fch1' :                    1926.26953125
         b'foff' :           -2.835503418452676e-06
   b'machine_id' :                               20
        b'nbits' :                               32
       b'nchans' :                        318230528
         b'nifs' :                                1
  b'source_name' :                      b'HIP56445'
      b'src_dej' :                     3d03m34.006s
      b'src_raj' :                    11h34m21.699s
 b'telescope_id' :                                6
        b'tsamp' :                     17.986224128
       b'tstart' :                57532.11040509259
     b'za_start' :                              0.0

Num ints in file :                               16
      File shape :               (16,

/Users/yuhongc/PycharmProjects/BSRC_Env/lib/python3.6/site-packages/blimpy/file_wrapper.py:326: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  self.h5 = h5py.File(self.filename)


In [ ]:
wf = Waterfall(fri_obs, f_stop=1080, max_load=2)
wf.info()

In [3]:
channel_len = np.int(np.round(187.5/64/abs(wf.header[b'foff'])))
channel_len

1033216

In [ ]:
plt.figure()
wf.plot_waterfall(f_start=0, f_stop=1080)

In [ ]:
freqs, data = wf.grab_data(f_start=0, f_stop=1080)
print(data.shape)
freqs = freqs[::-1]
data = data[:,::-1]

In [ ]:
len(freqs)/channel_len

In [ ]:
data = data[:, :16 * 1033216]
freqs = freqs[:16 * 1033216]

integrated = np.sum(data, axis=0)/len(data)
for n in np.nonzero(integrated > 10**13):
    integrated[n] = (integrated[n-1] + integrated[n+1]) /2
plt.figure()
plt.plot(freqs, np.log(integrated))


In [ ]:
c = 6

window = data[:, channel_len*(c):channel_len*(c+1)]
plot_segment(np.log(window))

In [ ]:
window

In [ ]:
res = []
for chan in tqdm(range(15)):
    res.append([])
    window = data[:, channel_len*(chan):channel_len*(chan+1)]
    window_f = freqs[channel_len*(chan):channel_len*(chan+1)]
    for i in range(0, (len(window[0])//200*200), 100):
        test_data = window[:, i:i+200]
        s, p = norm_test(test_data)
        if p < 1e-25:
            res[chan].append((window_f[i], s, p))

In [ ]:
from multiprocessing import Pool
from time import time

import warnings
warnings.filterwarnings("ignore")


def threshold_hits(chan):
    res = list()
    window = data[:, channel_len*(chan):channel_len*(chan+1)]
    window_f = freqs[channel_len*(chan):channel_len*(chan+1)]
    for i in range(0, (len(window[0])//200*200), 100):
        test_data = window[:, i:i+200]
        s, p = norm_test(test_data)
        if p < 1e-25:
            res.append((window_f[i], s, p))
    return res

start = time()
with Pool(12) as p:
    chan_hits = p.map(threshold_hits, range(16))
end = time()
print(end-start)

In [ ]:
show_stamp_f(freqs, data, 1025.0100805927195)

In [ ]:
hits = [len(e) for e in chan_hits]
print(hits)
print(sum(hits))

In [ ]:
sorted_hits = sorted(res[7], key=lambda x: x[2])
sorted_hits

In [ ]:
top = [x[0] for x in sorted_hits[:20]]
top

In [ ]:
for i in top:
    print(i)
    show_stamp(window, i)

In [ ]:
res[:15]

In [ ]:
plt.figure()
plt.imshow(data[:, 8:8+128])

In [ ]:
with open("pfb512coef.txt", "r") as f:
    coef_file = f.read()

In [ ]:
lines = coef_file.splitlines()
filter_coefs = []
for line in lines:
    filter_coefs.append(float(line))

In [ ]:
coefs = np.array(filter_coefs)/2**17

In [ ]:
plt.figure()
plt.plot(coefs)

In [ ]:
from numpy import fft
l = 2**16
f = fft.fft(coefs, l)
plt.figure()
plt.plot(np.log(np.abs(f)**2))

In [ ]:
stacked = np.reshape(integrated, (15, 1033216))
plt.figure()
for i in range(15):
    plt.plot(np.log(stacked[i]))

In [ ]:
rebined = rebin(stacked, 1, 64)

In [ ]:
plt.figure()
plt.imshow(np.log(rebined), **plt_args)

In [ ]:
model_shape = np.sum(stacked, axis=0)/15
model_shape[model_shape > 10**10] = np.mean(model_shape)

In [ ]:
plt.figure()
plt.plot(model_shape)

In [4]:
h5_file = h5py.File(fri_obs, "r")
a = da.from_array(h5_file["data"], chunks=(2, 1, channel_len*14))
a = a
a

,Array,Chunk
Bytes,20.37 GB,115.72 MB
Shape,"(16, 1, 318230528)","(2, 1, 14465024)"
Count,177 Tasks,176 Chunks
Type,float32,numpy.ndarray


In [ ]:
start = time()
da.to_npy_stack("original_a", a, axis=2)
end = time()
print(f"{end-start}")

distributed.comm.inproc - WARNING - Closing dangling queue in <InProc  local=inproc://10.0.0.32/43036/20 remote=inproc://10.0.0.32/43036/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc  local=inproc://10.0.0.32/43036/21 remote=inproc://10.0.0.32/43036/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc  local=inproc://10.0.0.32/43036/22 remote=inproc://10.0.0.32/43036/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc  local=inproc://10.0.0.32/43036/23 remote=inproc://10.0.0.32/43036/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc  local=inproc://10.0.0.32/43036/24 remote=inproc://10.0.0.32/43036/5>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc  local=inproc://10.0.0.32/43036/25 remote=inproc://10.0.0.32/43036/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc  local=inproc://10.0.0.32/43036/26 remote=inproc://10.0.0.32/43036/1>
distributed.worker -

distributed.comm.inproc - WARNING - Closing dangling queue in <InProc  local=inproc://10.0.0.32/43036/33 remote=inproc://10.0.0.32/43036/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc  local=inproc://10.0.0.32/43036/34 remote=inproc://10.0.0.32/43036/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc  local=inproc://10.0.0.32/43036/35 remote=inproc://10.0.0.32/43036/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc  local=inproc://10.0.0.32/43036/36 remote=inproc://10.0.0.32/43036/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc  local=inproc://10.0.0.32/43036/37 remote=inproc://10.0.0.32/43036/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc  local=inproc://10.0.0.32/43036/38 remote=inproc://10.0.0.32/43036/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc  local=inproc://10.0.0.32/43036/39 remote=inproc://10.0.0.32/43036/5>
distributed.worker -

distributed.comm.inproc - WARNING - Closing dangling queue in <InProc  local=inproc://10.0.0.32/43036/40 remote=inproc://10.0.0.32/43036/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc  local=inproc://10.0.0.32/43036/41 remote=inproc://10.0.0.32/43036/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc  local=inproc://10.0.0.32/43036/42 remote=inproc://10.0.0.32/43036/1>
distributed.worker - ERROR - Handle missing dep failed, retrying
Traceback (most recent call last):
  File "/Users/yuhongc/PycharmProjects/BSRC_Env/lib/python3.6/site-packages/distributed/comm/core.py", line 221, in connect
    _raise(error)
  File "/Users/yuhongc/PycharmProjects/BSRC_Env/lib/python3.6/site-packages/distributed/comm/core.py", line 204, in _raise
    raise IOError(msg)
OSError: Timed out trying to connect to 'inproc://10.0.0.32/43036/1' after 10 s: connect() didn't finish in time

During handling of the above exception, another exception occurred:

Traceback

distributed.worker - ERROR - Handle missing dep failed, retrying
Traceback (most recent call last):
  File "/Users/yuhongc/PycharmProjects/BSRC_Env/lib/python3.6/site-packages/distributed/comm/core.py", line 221, in connect
    _raise(error)
  File "/Users/yuhongc/PycharmProjects/BSRC_Env/lib/python3.6/site-packages/distributed/comm/core.py", line 204, in _raise
    raise IOError(msg)
OSError: Timed out trying to connect to 'inproc://10.0.0.32/43036/1' after 10 s: connect() didn't finish in time

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/yuhongc/PycharmProjects/BSRC_Env/lib/python3.6/site-packages/distributed/worker.py", line 2019, in handle_missing_dep
    who_has = await self.scheduler.who_has(keys=list(deps))
  File "/Users/yuhongc/PycharmProjects/BSRC_Env/lib/python3.6/site-packages/distributed/core.py", line 735, in send_recv_from_rpc
    comm = await self.pool.connect(self.addr)
  File "/Users/yuhongc/Py

distributed.comm.inproc - WARNING - Closing dangling queue in <InProc  local=inproc://10.0.0.32/43036/67 remote=inproc://10.0.0.32/43036/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc  local=inproc://10.0.0.32/43036/68 remote=inproc://10.0.0.32/43036/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc  local=inproc://10.0.0.32/43036/69 remote=inproc://10.0.0.32/43036/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc  local=inproc://10.0.0.32/43036/70 remote=inproc://10.0.0.32/43036/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc  local=inproc://10.0.0.32/43036/71 remote=inproc://10.0.0.32/43036/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc  local=inproc://10.0.0.32/43036/72 remote=inproc://10.0.0.32/43036/1>
distributed.worker - ERROR - Handle missing dep failed, retrying
Traceback (most recent call last):
  File "/Users/yuhongc/PycharmProjects/BSRC_Env/lib/python3.

...

distributed.worker - ERROR - Handle missing dep failed, retrying
Traceback (most recent call last):
  File "/Users/yuhongc/PycharmProjects/BSRC_Env/lib/python3.6/site-packages/distributed/comm/core.py", line 221, in connect
    _raise(error)
  File "/Users/yuhongc/PycharmProjects/BSRC_Env/lib/python3.6/site-packages/distributed/comm/core.py", line 204, in _raise
    raise IOError(msg)
OSError: Timed out trying to connect to 'inproc://10.0.0.32/43036/1' after 10 s: connect() didn't finish in time

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/yuhongc/PycharmProjects/BSRC_Env/lib/python3.6/site-packages/distributed/worker.py", line 2019, in handle_missing_dep
    who_has = await self.scheduler.who_has(keys=list(deps))
  File "/Users/yuhongc/PycharmProjects/BSRC_Env/lib/python3.6/site-packages/distributed/core.py", line 735, in send_recv_from_rpc
    comm = await self.pool.connect(self.addr)
  File "/Users/yuhongc/Py

distributed.worker - ERROR - Worker stream died during communication: inproc://10.0.0.32/43036/4
Traceback (most recent call last):
  File "/Users/yuhongc/PycharmProjects/BSRC_Env/lib/python3.6/site-packages/distributed/comm/core.py", line 221, in connect
    _raise(error)
  File "/Users/yuhongc/PycharmProjects/BSRC_Env/lib/python3.6/site-packages/distributed/comm/core.py", line 204, in _raise
    raise IOError(msg)
OSError: Timed out trying to connect to 'inproc://10.0.0.32/43036/4' after 10 s: connect() didn't finish in time

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/yuhongc/PycharmProjects/BSRC_Env/lib/python3.6/site-packages/distributed/worker.py", line 1881, in gather_dep
    self.rpc, deps, worker, who=self.address
  File "/Users/yuhongc/PycharmProjects/BSRC_Env/lib/python3.6/site-packages/distributed/worker.py", line 3099, in get_data_from_worker
    comm = await rpc.connect(worker)
  File "/Users/yuhon

distributed.worker - ERROR - Handle missing dep failed, retrying
Traceback (most recent call last):
  File "/Users/yuhongc/PycharmProjects/BSRC_Env/lib/python3.6/site-packages/distributed/comm/core.py", line 221, in connect
    _raise(error)
  File "/Users/yuhongc/PycharmProjects/BSRC_Env/lib/python3.6/site-packages/distributed/comm/core.py", line 204, in _raise
    raise IOError(msg)
OSError: Timed out trying to connect to 'inproc://10.0.0.32/43036/1' after 10 s: connect() didn't finish in time

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/yuhongc/PycharmProjects/BSRC_Env/lib/python3.6/site-packages/distributed/worker.py", line 2019, in handle_missing_dep
    who_has = await self.scheduler.who_has(keys=list(deps))
  File "/Users/yuhongc/PycharmProjects/BSRC_Env/lib/python3.6/site-packages/distributed/core.py", line 735, in send_recv_from_rpc
    comm = await self.pool.connect(self.addr)
  File "/Users/yuhongc/Py

distributed.comm.inproc - WARNING - Closing dangling queue in <InProc  local=inproc://10.0.0.32/43036/210 remote=inproc://10.0.0.32/43036/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc  local=inproc://10.0.0.32/43036/211 remote=inproc://10.0.0.32/43036/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc  local=inproc://10.0.0.32/43036/212 remote=inproc://10.0.0.32/43036/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc  local=inproc://10.0.0.32/43036/213 remote=inproc://10.0.0.32/43036/1>
tornado.application - ERROR - Exception in callback functools.partial(<function wrap.<locals>.null_wrapper at 0x3168c97b8>, <Task finished coro=<Worker.handle_missing_dep() done, defined at /Users/yuhongc/PycharmProjects/BSRC_Env/lib/python3.6/site-packages/distributed/worker.py:1996> exception=OSError("Timed out trying to connect to 'inproc://10.0.0.32/43036/1' after 10 s: Timed out trying to connect to 'inproc://10.0.0.32/43036/1'

distributed.worker - ERROR - Handle missing dep failed, retrying
Traceback (most recent call last):
  File "/Users/yuhongc/PycharmProjects/BSRC_Env/lib/python3.6/site-packages/distributed/comm/core.py", line 221, in connect
    _raise(error)
  File "/Users/yuhongc/PycharmProjects/BSRC_Env/lib/python3.6/site-packages/distributed/comm/core.py", line 204, in _raise
    raise IOError(msg)
OSError: Timed out trying to connect to 'inproc://10.0.0.32/43036/1' after 10 s: connect() didn't finish in time

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/yuhongc/PycharmProjects/BSRC_Env/lib/python3.6/site-packages/distributed/worker.py", line 2019, in handle_missing_dep
    who_has = await self.scheduler.who_has(keys=list(deps))
  File "/Users/yuhongc/PycharmProjects/BSRC_Env/lib/python3.6/site-packages/distributed/core.py", line 735, in send_recv_from_rpc
    comm = await self.pool.connect(self.addr)
  File "/Users/yuhongc/Py

In [ ]:
start = time()
means = da.mean(a, axis=2)
# means_data = means.compute()
# means_data
end = time()

print(f"{end-start}")

In [ ]:
means = da.from_array(np.reshape(means_data, (16,1,1)))
means

In [ ]:
a

In [ ]:
308/14

In [ ]:
normalized_a = da.divide(a.T, means.T).T
normalized_a

In [ ]:
da.to_npy_stack('normalized_a/', normalized_a, axis=2)

In [ ]:
print(normalized_a.shape)
print(normalized_a.chunks)

In [ ]:
start = time()
normalized_a_data = normalized_a.compute()
end = time()
print(f"{end-start}")

In [ ]:
start = time()

original_a_data = a[:, 0, :16 * 1033216].compute()

end = time()
print(f"{end-start}")

In [ ]:
read_a = da.from_array(original_a_data)
read_and_normalize = da.divide(read_a, means_data)

start = time()
normalized_a_data = read_and_normalize.compute()
end = time()
print(f"{end-start}")

# 1.6420118808746338

In [ ]:
del normalized_a_data

In [ ]:
start = time()

numpy_normalized_a_data = original_a_data / means_data

end = time()
print(f"{end-start}")

# 0.6659681797027588

In [ ]:
del numpy_normalized_a_data

In [ ]:
numpy_normalized_a_data.shape

In [ ]:
plot_segment(normalized_a_data)

In [ ]:
plot_segment(original_a_data)

In [ ]:
print(normalized_a_data.shape == numpy_normalized_a_data.shape)
print(np.allclose(normalized_a_data, numpy_normalized_a_data))